In [ ]:
# Import the package
from running_results import (
    RaceDataFetcher,
    CSVRaceData,
    TimeConverter,
    ColumnStandardizer,
    NameParser,
    RaceDataTransformer,
    KentigernPlot,
    RacePlotter,
    RaceStatistics
)
import pandas as pd

## Example 1: Loading and Analyzing Existing CSV Data

In [ ]:
# Load existing race data
loader = CSVRaceData("edinburgh-marathon-2024.csv")
data = loader.fetch()

# Show the data
data.head()

## Example 2: Data Transformation

In [ ]:
# If you need to convert times
# data['Chip Time (seconds)'] = data['Chip Time'].apply(TimeConverter.to_seconds)
# data['Chip Time (minutes)'] = data['Chip Time (seconds)'] / 60

# The Edinburgh data already has this, so we can skip this step
print(f"Total runners: {len(data)}")
print(f"Columns: {list(data.columns)}")

## Example 3: Creating Visualizations with Kentigern Style

In [ ]:
# Create a race plotter
plotter = RacePlotter(use_kentigern_style=True)

# Plot finish time distribution
fig = plotter.plot_finish_time_distribution(
    data,
    time_column='Chip Time (minutes)',
    bins=range(120, 500, 1)
)

In [ ]:
# Plot gender comparison
fig = plotter.plot_gender_comparison(
    data,
    time_column='Chip Time (minutes)',
    gender_column='Category',
    bins=range(120, 420, 1)
)

In [ ]:
# Plot cumulative distribution
fig = plotter.plot_cumulative_distribution(
    data,
    time_column='Chip Time (minutes)',
    bins=range(120, 420, 1)
)

## Example 4: Statistical Analysis

In [ ]:
# Create statistics object
stats = RaceStatistics(data)

# Get summary statistics
print("Summary Statistics:")
print(stats.summary_statistics('Chip Time (minutes)'))

In [ ]:
# Get quantiles
print("\nQuantiles:")
print(stats.quantiles('Chip Time (minutes)'))

In [ ]:
# Percentile table
print("\nPercentile Table:")
print(stats.percentile_table('Chip Time (minutes)'))

In [ ]:
# Gender comparison
print("\nGender Comparison:")
print(stats.gender_comparison(
    gender_column='Category',
    time_column='Chip Time (minutes)'
))

In [ ]:
# Top 10 performers
print("\nTop 10 Fastest Finishers:")
print(stats.top_performers(n=10, time_column='Chip Time (minutes)'))

## Example 5: Using the KentigernPlot Style Directly

In [ ]:
# For custom plots, use the KentigernPlot context manager
import numpy as np
from matplotlib.mlab import GaussianKDE

with KentigernPlot(1, 1, figsize=(5, 3)) as fig:
    ax = fig.axes[0]
    
    # Create custom plot
    times = data['Chip Time (minutes)'].dropna()
    bins = range(120, 420, 1)
    
    # Plot histogram
    ax.hist(times, bins=bins, alpha=0.5, color='#79C4F2')
    
    # Add labels
    ax.set_xlabel("Finish Time (minutes)")
    ax.set_ylabel("Count")
    ax.set_title("Edinburgh Marathon 2024 - Custom Plot")

## Example 6: Working with Tinto Hill Race Data

In [ ]:
# Load multiple years of Tinto data
import glob

tinto_files = glob.glob("tinto/*.csv")
tinto_data = []

for file in sorted(tinto_files):
    try:
        year = file.split('/')[-1].split('.')[0]
        df = pd.read_csv(file)
        df['year'] = int(year)
        tinto_data.append(df)
    except:
        pass

if tinto_data:
    all_tinto = pd.concat(tinto_data, ignore_index=True)
    print(f"Loaded {len(all_tinto)} results from {len(tinto_data)} years")
    print(f"Years: {sorted(all_tinto['year'].unique())}")

In [ ]:
# Convert Tinto finish times to minutes (they're in format HH:MM:SS or MM:SS)
if tinto_data:
    all_tinto['FinishTime (minutes)'] = all_tinto['FinishTime'].apply(TimeConverter.to_minutes)
    
    # Statistics for Tinto
    tinto_stats = RaceStatistics(all_tinto)
    print("Tinto Hill Race Summary:")
    print(tinto_stats.summary_statistics('FinishTime (minutes)'))

In [ ]:
# Year-over-year comparison for Tinto
if tinto_data:
    print("\nYear-over-Year Comparison:")
    print(tinto_stats.year_over_year_comparison(
        year_column='year',
        time_column='FinishTime (minutes)',
        stat='median'
    ))

In [ ]:
# Plot Tinto finish time distribution
if tinto_data:
    plotter.plot_finish_time_distribution(
        all_tinto,
        time_column='FinishTime (minutes)',
        bins=range(30, 150, 1)
    )

## Summary

The `running_results` package provides:

1. **Data Fetching**: Easily fetch race results from web sources or load from CSV
2. **Transformation**: Standardize columns, parse names, convert times
3. **Visualization**: Beautiful, consistent plots with the Kentigern style
4. **Statistics**: Comprehensive analysis tools for race data

All the logic from your analysis notebooks is now in a reusable, extensible framework!